## Solutions for OpenAI ROS Project

<img src="../img/robotignite_logo_text.png"/>

## Index: 

* <a href="#SolutionStep1">Solution Step 1</a>
* <a href="#SolutionStep2">Solution Step 2</a>
* <a href="#SolutionStep3">Solution Step 3</a>

## Solution Step 1: Create Robot Environment <p id="SolutionStep1"></p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Python File: hopper_robot_env.py** </p>

In [ ]:
import numpy
import rospy
import time
from openai_ros import robot_gazebo_env
from gazebo_msgs.msg import ContactsState
from sensor_msgs.msg import Imu
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Point, Quaternion, Vector3
from sensor_msgs.msg import JointState
from std_msgs.msg import Float64



class HopperEnv(robot_gazebo_env.RobotGazeboEnv):
    """Superclass for all HopperEnv environments.
    """

    def __init__(self):
        """
        Initializes a new HopperEnv environment.
        
        To check any topic we need to have the simulations running, we need to do two things:
        1) Unpause the simulation: without that th stream of data doesnt flow. This is for simulations
        that are pause for whatever the reason
        2) If the simulation was running already for some reason, we need to reset the controlers.
        This has to do with the fact that some plugins with tf, dont understand the reset of the simulation
        and need to be reseted to work properly.
        
        The Sensors: The sensors accesible are the ones considered usefull for AI learning.
        
        Sensor Topic List:
        * /drone/down_camera/image_raw: RGB Camera facing down.
        * /drone/front_camera/image_raw: RGB Camera facing front.
        * /drone/imu: IMU of the drone giving acceleration and orientation relative to world.
        * /drone/sonar: Sonar readings facing front
        * /drone/gt_pose: Get position and orientation in Global space
        * /drone/gt_vel: Get the linear velocity , the angular doesnt record anything.
        
        Actuators Topic List: 
        * /cmd_vel: Move the Drone Around when you have taken off.
        * /drone/takeoff: Publish into it to take off
        * /drone/land: Publish to make ParrotDrone Land
        
        Args:
        """
        rospy.logdebug("Start HopperEnv INIT...")
        # Variables that we give through the constructor.
        # None in this case

        # Internal Vars
        # Doesnt have any accesibles
        self.controllers_list = ['joint_state_controller',
                                 'haa_joint_position_controller',
                                 'hfe_joint_position_controller',
                                 'kfe_joint_position_controller']

        # It doesnt use namespace
        self.robot_name_space = "monoped"

        # We launch the init function of the Parent Class robot_gazebo_env.RobotGazeboEnv
        super(HopperEnv, self).__init__(controllers_list=self.controllers_list,
                                            robot_name_space=self.robot_name_space,
                                            reset_controls=False,
                                            start_init_physics_parameters=False,
                                            reset_world_or_sim="WORLD")



        rospy.logdebug("HopperEnv unpause1...")
        self.gazebo.unpauseSim()
        #self.controllers_object.reset_controllers()
        
        self._check_all_systems_ready()


        # We Start all the ROS related Subscribers and publishers
        rospy.Subscriber("/odom", Odometry, self._odom_callback)
        # We use the IMU for orientation and linearacceleration detection
        rospy.Subscriber("/monoped/imu/data", Imu, self._imu_callback)
        # We use it to get the contact force, to know if its in the air or stumping too hard.
        rospy.Subscriber("/lowerleg_contactsensor_state", ContactsState, self._contact_callback)
        # We use it to get the joints positions and calculate the reward associated to it
        rospy.Subscriber("/monoped/joint_states", JointState, self._joints_state_callback)
        

        self.publishers_array = []
        self._haa_joint_pub = rospy.Publisher('/monoped/haa_joint_position_controller/command', Float64, queue_size=1)
        self._hfe_joint_pub = rospy.Publisher('/monoped/hfe_joint_position_controller/command', Float64, queue_size=1)
        self._kfe_joint_pub = rospy.Publisher('/monoped/kfe_joint_position_controller/command', Float64, queue_size=1)
        
        self.publishers_array.append(self._haa_joint_pub)
        self.publishers_array.append(self._hfe_joint_pub)
        self.publishers_array.append(self._kfe_joint_pub)

        self._check_all_publishers_ready()

        self.gazebo.pauseSim()
        
        rospy.logdebug("Finished HopperEnv INIT...")

    # Methods needed by the RobotGazeboEnv
    # ----------------------------
    

    def _check_all_systems_ready(self):
        """
        Checks that all the sensors, publishers and other simulation systems are
        operational.
        """
        rospy.logdebug("HopperEnv check_all_systems_ready...")
        self._check_all_sensors_ready()
        rospy.logdebug("END HopperEnv _check_all_systems_ready...")
        return True


    # CubeSingleDiskEnv virtual methods
    # ----------------------------

    def _check_all_sensors_ready(self):
        rospy.logdebug("START ALL SENSORS READY")
        self._check_odom_ready()
        self._check_imu_ready()
        self._check_lowerleg_contactsensor_state_ready()
        self._check_joint_states_ready()
        rospy.logdebug("ALL SENSORS READY")

        
    def _check_odom_ready(self):
        self.odom = None
        rospy.logdebug("Waiting for /odom to be READY...")
        while self.odom is None and not rospy.is_shutdown():
            try:
                self.odom = rospy.wait_for_message("/odom", Odometry, timeout=1.0)
                rospy.logdebug("Current /odom READY=>")

            except:
                rospy.logerr("Current /odom not ready yet, retrying for getting odom")
        return self.odom
        
        
    def _check_imu_ready(self):
        self.imu = None
        rospy.logdebug("Waiting for /monoped/imu/data to be READY...")
        while self.imu is None and not rospy.is_shutdown():
            try:
                self.imu = rospy.wait_for_message("/monoped/imu/data", Imu, timeout=1.0)
                rospy.logdebug("Current /monoped/imu/data READY=>")

            except:
                rospy.logerr("Current /monoped/imu/data not ready yet, retrying for getting imu")
        return self.imu

        
    def _check_lowerleg_contactsensor_state_ready(self):
        self.lowerleg_contactsensor_state = None
        rospy.logdebug("Waiting for /lowerleg_contactsensor_state to be READY...")
        while self.lowerleg_contactsensor_state is None and not rospy.is_shutdown():
            try:
                self.lowerleg_contactsensor_state = rospy.wait_for_message("/lowerleg_contactsensor_state", ContactsState, timeout=1.0)
                rospy.logdebug("Current /lowerleg_contactsensor_state READY=>")

            except:
                rospy.logerr("Current /lowerleg_contactsensor_state not ready yet, retrying for getting lowerleg_contactsensor_state")
        return self.lowerleg_contactsensor_state
        
    def _check_joint_states_ready(self):
        self.joint_states = None
        rospy.logdebug("Waiting for /monoped/joint_states to be READY...")
        while self.joint_states is None and not rospy.is_shutdown():
            try:
                self.joint_states = rospy.wait_for_message("/monoped/joint_states", JointState, timeout=1.0)
                rospy.logdebug("Current /monoped/joint_states READY=>")

            except:
                rospy.logerr("Current /monoped/joint_states not ready yet, retrying for getting joint_states")
        return self.joint_states



    def _odom_callback(self, data):
        self.odom = data
    
    def _imu_callback(self, data):
        self.imu = data
        
    def _contact_callback(self, data):
        self.lowerleg_contactsensor_state = data
        
    def _joints_state_callback(self, data):
        self.joint_states = data


    def _check_all_publishers_ready(self):
        """
        Checks that all the publishers are working
        :return:
        """
        rospy.logdebug("START ALL SENSORS READY")
        for publisher_object in self.publishers_array:
            self._check_pub_connection(publisher_object)
        rospy.logdebug("ALL SENSORS READY")

    def _check_pub_connection(self, publisher_object):

        rate = rospy.Rate(10)  # 10hz
        while publisher_object.get_num_connections() == 0 and not rospy.is_shutdown():
            rospy.logdebug("No susbribers to publisher_object yet so we wait and try again")
            try:
                rate.sleep()
            except rospy.ROSInterruptException:
                # This is to avoid error when world is rested, time when backwards.
                pass
        rospy.logdebug("publisher_object Publisher Connected")

        rospy.logdebug("All Publishers READY")
        
    
    # Methods that the TrainingEnvironment will need to define here as virtual
    # because they will be used in RobotGazeboEnv GrandParentClass and defined in the
    # TrainingEnvironment.
    # ----------------------------
    def _set_init_pose(self):
        """Sets the Robot in its init pose
        """
        raise NotImplementedError()
    
    def _init_env_variables(self):
        """Inits variables needed to be initialised each time we reset at the start
        of an episode.
        """
        raise NotImplementedError()

    def _compute_reward(self, observations, done):
        """Calculates the reward to give based on the observations given.
        """
        raise NotImplementedError()

    def _set_action(self, action):
        """Applies the given action to the simulation.
        """
        raise NotImplementedError()

    def _get_obs(self):
        raise NotImplementedError()

    def _is_done(self, observations):
        """Checks if episode done based on observations given.
        """
        raise NotImplementedError()
        
    # Methods that the TrainingEnvironment will need.
    # ----------------------------
    def move_joints(self, joints_array, epsilon=0.05, update_rate=10):
        """
        It will move the Hopper Joints to the given Joint_Array values
        """
        i = 0
        for publisher_object in self.publishers_array:
          joint_value = Float64()
          joint_value.data = joints_array[i]
          rospy.logdebug("JointsPos>>"+str(joint_value))
          publisher_object.publish(joint_value)
          i += 1
        self.wait_time_for_execute_movement(joints_array, epsilon, update_rate)
    
    def wait_time_for_execute_movement(self, joints_array, epsilon, update_rate):
        """
        We wait until Joints are where we asked them to be based on the joints_states
        :param joints_array:Joints Values in radians of each of the three joints of hopper leg.
        :param epsilon: Error acceptable in odometry readings.
        :param update_rate: Rate at which we check the joint_states.
        :return:
        """
        rospy.logdebug("START wait_until_twist_achieved...")
        
        rate = rospy.Rate(update_rate)
        start_wait_time = rospy.get_rostime().to_sec()
        end_wait_time = 0.0

        rospy.logdebug("Desired JointsState>>" + str(joints_array))
        rospy.logdebug("epsilon>>" + str(epsilon))
        
        while not rospy.is_shutdown():
            current_joint_states = self._check_joint_states_ready()
            
            values_to_check = [ current_joint_states.position[0],
                                current_joint_states.position[1],
                                current_joint_states.position[2]]
            
            vel_values_are_close = self.check_array_similar(joints_array,values_to_check,epsilon)
            
            if vel_values_are_close:
                rospy.logdebug("Reached JointStates!")
                end_wait_time = rospy.get_rostime().to_sec()
                break
            rospy.logdebug("Not there yet, keep waiting...")
            rate.sleep()
        delta_time = end_wait_time- start_wait_time
        rospy.logdebug("[Wait Time=" + str(delta_time)+"]")
        
        rospy.logdebug("END wait_until_jointstate_achieved...")
        
        return delta_time
    
    def check_array_similar(self,ref_value_array,check_value_array,epsilon):
        """
        It checks if the check_value id similar to the ref_value
        """
        rospy.logdebug("ref_value_array="+str(ref_value_array))
        rospy.logdebug("check_value_array="+str(check_value_array))
        return numpy.allclose(ref_value_array, check_value_array, atol=epsilon)
    

    def get_odom(self):
        return self.odom
    
    def get_imu(self):
        return self.imu
    
    def get_lowerleg_contactsensor_state(self):
        return self.lowerleg_contactsensor_state
        
    def get_joint_states(self):
        return self.joint_states

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Python File: hopper_robot_env.py** </p>

## Solution Step 2: Create Task Environment <p id="SolutionStep2"></p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Python File: hopper_task_env.py** </p>

In [ ]:
import rospy
import numpy
from gym import spaces
import hopper_robot_env
from gym.envs.registration import register
from geometry_msgs.msg import Point
from geometry_msgs.msg import Vector3
from tf.transformations import euler_from_quaternion

timestep_limit_per_episode = 10000 # Can be any Value

register(
        id='HopperStayUp-v0',
        entry_point='hopper_task_env:HopperStayUpEnv',
        timestep_limit=timestep_limit_per_episode,
    )

class HopperStayUpEnv(hopper_robot_env.HopperEnv):
    def __init__(self):
        """
        Make Hopper Learn how to Stay Up indefenitly
        """
        
        # Only variable needed to be set here
        """
        For this version, we consider 6 actions
        1-2) Increment/Decrement haa_joint
        3-4) Increment/Decrement hfe_joint
        5-6) Increment/Decrement kfe_joint
        """
        rospy.logdebug("Start HopperStayUpEnv INIT...")
        number_actions = rospy.get_param('/monoped/n_actions')
        self.action_space = spaces.Discrete(number_actions)
        
        # We set the reward range, which is not compulsory but here we do it.
        self.reward_range = (-numpy.inf, numpy.inf)
        
        
        # Actions and Observations
        
        self.init_joint_states = Vector3()
        self.init_joint_states.x = rospy.get_param('/monoped/init_joint_states/haa_joint')
        self.init_joint_states.y = rospy.get_param('/monoped/init_joint_states/hfe_joint')
        self.init_joint_states.z = rospy.get_param('/monoped/init_joint_states/kfe_joint')
        
        
        # Get Desired Point to Get
        self.desired_point = Point()
        self.desired_point.x = rospy.get_param("/monoped/desired_point/x")
        self.desired_point.y = rospy.get_param("/monoped/desired_point/y")
        self.desired_point.z = rospy.get_param("/monoped/desired_point/z")
        self.accepted_error_in_des_pos = rospy.get_param("/monoped/accepted_error_in_des_pos")
        
        self.desired_yaw = rospy.get_param("/monoped/desired_yaw")
        
        self.joint_increment_value = rospy.get_param("/monoped/joint_increment_value")
        self.accepted_joint_error = rospy.get_param("/monoped/accepted_joint_error")
        self.update_rate = rospy.get_param("/monoped/update_rate")

        self.dec_obs = rospy.get_param("/monoped/number_decimals_precision_obs")
        
        self.desired_force = rospy.get_param("/monoped/desired_force")
        
        self.max_x_pos = rospy.get_param("/monoped/max_x_pos")
        self.max_y_pos = rospy.get_param("/monoped/max_y_pos")
        
        self.min_height = rospy.get_param("/monoped/min_height")
        self.max_height = rospy.get_param("/monoped/max_height")
        
        self.distance_from_desired_point_max = rospy.get_param("/monoped/distance_from_desired_point_max")
        
        self.max_incl_roll = rospy.get_param("/monoped/max_incl")
        self.max_incl_pitch = rospy.get_param("/monoped/max_incl")
        self.max_contact_force = rospy.get_param("/monoped/max_contact_force")
        
        self.maximum_haa_joint = rospy.get_param("/monoped/max_incl")
        self.maximum_hfe_joint = rospy.get_param("/monoped/max_incl")
        self.maximum_kfe_joint = rospy.get_param("/monoped/max_incl")
        self.min_kfe_joint = rospy.get_param("/monoped/max_incl")
        
        # We place the Maximum and minimum values of observations

        high = numpy.array([self.distance_from_desired_point_max,
                            self.max_incl_roll,
                            self.max_incl_pitch,
                            3.14,
                            self.max_contact_force,
                            self.maximum_haa_joint,
                            self.maximum_hfe_joint,
                            self.maximum_kfe_joint,
                            self.max_x_pos,
                            self.max_y_pos,
                            self.max_height
                            ])
                                        
        low = numpy.array([ 0.0,
                            -1*self.max_incl_roll,
                            -1*self.max_incl_pitch,
                            -1*3.14,
                            0.0,
                            self.maximum_haa_joint,
                            self.maximum_hfe_joint,
                            self.min_kfe_joint,
                            -1*self.max_x_pos,
                            -1*self.max_y_pos,
                            self.min_height
                            ])

        
        self.observation_space = spaces.Box(low, high)
        
        rospy.logdebug("ACTION SPACES TYPE===>"+str(self.action_space))
        rospy.logdebug("OBSERVATION SPACES TYPE===>"+str(self.observation_space))
        
        # Rewards
        self.weight_joint_position = rospy.get_param("/monoped/rewards_weight/weight_joint_position")
        self.weight_contact_force = rospy.get_param("/monoped/rewards_weight/weight_contact_force")
        self.weight_orientation = rospy.get_param("/monoped/rewards_weight/weight_orientation")
        self.weight_distance_from_des_point = rospy.get_param("/monoped/rewards_weight/weight_distance_from_des_point")
        
        self.alive_reward =rospy.get_param("/monoped/alive_reward")
        self.done_reward =rospy.get_param("/monoped/done_reward")

        # Here we will add any init functions prior to starting the MyRobotEnv
        super(HopperStayUpEnv, self).__init__()
        
        rospy.logdebug("END HopperStayUpEnv INIT...")

    def _set_init_pose(self):
        """
        Sets the Robot in its init linear and angular speeds
        and lands the robot. Its preparing it to be reseted in the world.
        """

        joints_array = [self.init_joint_states.x,
                        self.init_joint_states.y,
                        self.init_joint_states.z]
        
        self.move_joints(   joints_array,
                            epsilon=self.accepted_joint_error,
                            update_rate=self.update_rate)

        return True


    def _init_env_variables(self):
        """
        Inits variables needed to be initialised each time we reset at the start
        of an episode.
        :return:
        """

        # For Info Purposes
        self.cumulated_reward = 0.0
        # We get the initial pose to mesure the distance from the desired point.
        odom = self.get_odom()
        self.previous_distance_from_des_point = self.get_distance_from_desired_point(odom.pose.pose.position)

        

    def _set_action(self, action):
        """
        It sets the joints of monoped based on the action integer given
        based on the action number given.
        :param action: The action integer that sets what movement to do next.
        """
        
        rospy.logdebug("Start Set Action ==>"+str(action))
       
        # We get current Joints values
        joint_states = self.get_joint_states()
        joint_states_position = joint_states.position
        rospy.logdebug("get_action_to_position>>>"+str(joint_states_position))
        
        action_position = [0.0,0.0,0.0]
        if action == 0: #Increment haa_joint
            action_position[0] = joint_states_position[0] + self.joint_increment_value
            action_position[1] = joint_states_position[1]
            action_position[2] = joint_states_position[2]
        elif action == 1: #Decrement haa_joint
            action_position[0] = joint_states_position[0] - self.joint_increment_value
            action_position[1] = joint_states_position[1]
            action_position[2] = joint_states_position[2]
        elif action == 2: #Increment hfe_joint
            action_position[0] = joint_states_position[0]
            action_position[1] = joint_states_position[1] + self.joint_increment_value
            action_position[2] = joint_states_position[2]
        elif action == 3: #Decrement hfe_joint
            action_position[0] = joint_states_position[0]
            action_position[1] = joint_states_position[1] - self.joint_increment_value
            action_position[2] = joint_states_position[2]
        elif action == 4: #Increment kfe_joint
            action_position[0] = joint_states_position[0]
            action_position[1] = joint_states_position[1]
            action_position[2] = joint_states_position[2] + self.joint_increment_value
        elif action == 5:  #Decrement kfe_joint
            action_position[0] = joint_states_position[0]
            action_position[1] = joint_states_position[1]
            action_position[2] = joint_states_position[2] - self.joint_increment_value

        
        # We tell monoped where to place its joints next
        self.move_joints(   action_position,
                            epsilon=self.accepted_joint_error,
                            update_rate=self.update_rate)
        
        rospy.logdebug("END Set Action ==>"+str(action))

    def _get_obs(self):
        """
        Here we define what sensor data defines our robots observations
        To know which Variables we have access to, we need to read the
        HopperEnv API DOCS
        Returns the state of the robot needed for OpenAI QLearn Algorithm
        The state will be defined by an array of the:
        1) distance from desired point in meters
        2) The pitch orientation in radians
        3) the Roll orientation in radians
        4) the Yaw orientation in radians
        5) Force in contact sensor in Newtons
        6-7-8) State of the 3 joints in radians
        9) Height of the Base

        observation = [distance_from_desired_point,
                 base_roll,
                 base_pitch,
                 base_yaw,
                 force_magnitude,
                 joint_states_haa,
                 joint_states_hfe,
                 joint_states_kfe,
                 height_base]
        :return: observation
        """
        rospy.logdebug("Start Get Observation ==>")
        
        distance_from_desired_point = self.get_distance_from_desired_point(self.desired_point)

        base_orientation = self.get_base_rpy()
        base_roll = base_orientation.x
        base_pitch = base_orientation.y
        base_yaw = base_orientation.z

        force_magnitude = self.get_contact_force_magnitude()

        joint_states = self.get_joint_states()
        joint_states_haa = joint_states.position[0]
        joint_states_hfe = joint_states.position[1]
        joint_states_kfe = joint_states.position[2]
        
        odom = self.get_odom()
        base_position = odom.pose.pose.position

        observation = []
        observation.append(round(distance_from_desired_point,self.dec_obs))
        observation.append(round(base_roll,self.dec_obs))
        observation.append(round(base_pitch,self.dec_obs))
        observation.append(round(base_yaw,self.dec_obs))
        observation.append(round(force_magnitude,self.dec_obs))
        observation.append(round(joint_states_haa,self.dec_obs))
        observation.append(round(joint_states_hfe,self.dec_obs))
        observation.append(round(joint_states_kfe,self.dec_obs))
        
        observation.append(round(base_position.x,self.dec_obs))
        observation.append(round(base_position.y,self.dec_obs))
        observation.append(round(base_position.z,self.dec_obs)) # height

        return observation
        

    def _is_done(self, observations):
        """
        We consider the episode done if:
        1) The Monopeds height is lower than a threshhold
        2) The Orientation is outside a threshold
        """
        
        
        height_base = observations[10]
        
        monoped_height_ok = self.monoped_height_ok(height_base)
        monoped_orientation_ok = self.monoped_orientation_ok()

        done = not(monoped_height_ok and monoped_orientation_ok)
        
        return done

    def _compute_reward(self, observations, done):
        """
        We Base the rewards in if its done or not and we base it on
        the joint poisition, effort, contact force, orientation and distance from desired point.
        :return:
        """
        
        joints_state_array = observations[5:8]
        r1 = self.calculate_reward_joint_position(joints_state_array, self.weight_joint_position)
        # Desired Force in Newtons, taken form idle contact with 9.81 gravity.
        
        force_magnitude = observations[4]
        r2 = self.calculate_reward_contact_force(force_magnitude, self.weight_contact_force)
        
        rpy_array = observations[1:4]
        r3 = self.calculate_reward_orientation(rpy_array, self.weight_orientation)
        
        
        current_position = Point()
        current_position.x = observations[8]
        current_position.y = observations[9]
        current_position.z = observations[10]
        r4 = self.calculate_reward_distance_from_des_point(current_position, self.weight_distance_from_des_point)

        # The sign depend on its function.
        total_reward = self.alive_reward - r1 - r2 - r3 - r4

        rospy.logdebug("###############")
        rospy.logdebug("alive_bonus=" + str(self.alive_reward))
        rospy.logdebug("r1 joint_position=" + str(r1))
        rospy.logdebug("r2 contact_force=" + str(r2))
        rospy.logdebug("r3 orientation=" + str(r3))
        rospy.logdebug("r4 distance=" + str(r4))
        rospy.logdebug("total_reward=" + str(total_reward))
        rospy.logdebug("###############")

        return total_reward


    # Internal TaskEnv Methods
    
    def is_in_desired_position(self,current_position, epsilon=0.05):
        """
        It return True if the current position is similar to the desired poistion
        """
        
        is_in_desired_pos = False
        
        
        x_pos_plus = self.desired_point.x + epsilon
        x_pos_minus = self.desired_point.x - epsilon
        y_pos_plus = self.desired_point.y + epsilon
        y_pos_minus = self.desired_point.y - epsilon
        
        x_current = current_position.x
        y_current = current_position.y
        
        x_pos_are_close = (x_current <= x_pos_plus) and (x_current > x_pos_minus)
        y_pos_are_close = (y_current <= y_pos_plus) and (y_current > y_pos_minus)
        
        is_in_desired_pos = x_pos_are_close and y_pos_are_close
        
        rospy.logdebug("###### IS DESIRED POS ? ######")
        rospy.logdebug("current_position"+str(current_position))
        rospy.logdebug("x_pos_plus"+str(x_pos_plus)+",x_pos_minus="+str(x_pos_minus))
        rospy.logdebug("y_pos_plus"+str(y_pos_plus)+",y_pos_minus="+str(y_pos_minus))
        rospy.logdebug("x_pos_are_close"+str(x_pos_are_close))
        rospy.logdebug("y_pos_are_close"+str(y_pos_are_close))
        rospy.logdebug("is_in_desired_pos"+str(is_in_desired_pos))
        rospy.logdebug("############")
        
        return is_in_desired_pos
    
    def is_inside_workspace(self,current_position):
        """
        Check if the monoped is inside the Workspace defined
        """
        is_inside = False

        rospy.logdebug("##### INSIDE WORK SPACE? #######")
        rospy.logdebug("XYZ current_position"+str(current_position))
        rospy.logdebug("work_space_x_max"+str(self.work_space_x_max)+",work_space_x_min="+str(self.work_space_x_min))
        rospy.logdebug("work_space_y_max"+str(self.work_space_y_max)+",work_space_y_min="+str(self.work_space_y_min))
        rospy.logdebug("work_space_z_max"+str(self.work_space_z_max)+",work_space_z_min="+str(self.work_space_z_min))
        rospy.logdebug("############")

        if current_position.x > self.work_space_x_min and current_position.x <= self.work_space_x_max:
            if current_position.y > self.work_space_y_min and current_position.y <= self.work_space_y_max:
                if current_position.z > self.work_space_z_min and current_position.z <= self.work_space_z_max:
                    is_inside = True
        
        return is_inside
        
    def sonar_detected_something_too_close(self, sonar_value):
        """
        Detects if there is something too close to the monoped front
        """
        rospy.logdebug("##### SONAR TOO CLOSE? #######")
        rospy.logdebug("sonar_value"+str(sonar_value)+",min_sonar_value="+str(self.min_sonar_value))
        rospy.logdebug("############")
        
        too_close = sonar_value < self.min_sonar_value
        
        return too_close
        
    def monoped_has_flipped(self,current_orientation):
        """
        Based on the orientation RPY given states if the monoped has flipped
        """
        has_flipped = True
        
        
        self.max_roll = rospy.get_param("/monoped/max_roll")
        self.max_pitch = rospy.get_param("/monoped/max_pitch")
        
        rospy.logdebug("#### HAS FLIPPED? ########")
        rospy.logdebug("RPY current_orientation"+str(current_orientation))
        rospy.logdebug("max_roll"+str(self.max_roll)+",min_roll="+str(-1*self.max_roll))
        rospy.logdebug("max_pitch"+str(self.max_pitch)+",min_pitch="+str(-1*self.max_pitch))
        rospy.logdebug("############")
        
        
        if current_orientation.x > -1*self.max_roll and current_orientation.x <= self.max_roll:
            if current_orientation.y > -1*self.max_pitch and current_orientation.y <= self.max_pitch:
                    has_flipped = False
        
        return has_flipped
        
    def get_distance_from_desired_point(self, current_position):
        """
        Calculates the distance from the current position to the desired point
        :param start_point:
        :return:
        """
        distance = self.get_distance_from_point(current_position,
                                                self.desired_point)
    
        return distance
    
    def get_distance_from_point(self, pstart, p_end):
        """
        Given a Vector3 Object, get distance from current position
        :param p_end:
        :return:
        """
        a = numpy.array((pstart.x, pstart.y, pstart.z))
        b = numpy.array((p_end.x, p_end.y, p_end.z))
    
        distance = numpy.linalg.norm(a - b)
    
        return distance
        
    def get_orientation_euler(self, quaternion_vector):
        # We convert from quaternions to euler
        orientation_list = [quaternion_vector.x,
                            quaternion_vector.y,
                            quaternion_vector.z,
                            quaternion_vector.w]
    
        roll, pitch, yaw = euler_from_quaternion(orientation_list)
        return roll, pitch, yaw
        

    def get_base_rpy(self):

        imu = self.get_imu()
        base_orientation = imu.orientation
        
        euler_rpy = Vector3()
        euler = euler_from_quaternion([ base_orientation.x,
                                        base_orientation.y,
                                        base_orientation.z,
                                        base_orientation.w]
                                        )
        euler_rpy.x = euler[0]
        euler_rpy.y = euler[1]
        euler_rpy.z = euler[2]
        
        return euler_rpy
        
    def get_contact_force_magnitude(self):
        """
        You will see that because the X axis is the one pointing downwards, it will be the one with
        higher value when touching the floor
        For a Robot of total mas of 0.55Kg, a gravity of 9.81 m/sec**2, Weight = 0.55*9.81=5.39 N
        Falling from around 5centimetres ( negligible height ), we register peaks around
        Fx = 7.08 N
        :return:
        """
        # We get the Contact Sensor data
        lowerleg_contactsensor_state = self.get_lowerleg_contactsensor_state()
        # We extract what we need that is only the total_wrench force
        contact_force = self.get_contact_force(lowerleg_contactsensor_state)
        # We create an array with each component XYZ
        contact_force_np = numpy.array((contact_force.x, contact_force.y, contact_force.z))
        # We calculate the magnitude of the Force Vector, array.
        force_magnitude = numpy.linalg.norm(contact_force_np)

        return force_magnitude
        
    def get_contact_force(self, lowerleg_contactsensor_state):
        """
        /lowerleg_contactsensor_state/states[0]/contact_positions ==> PointContact in World
        /lowerleg_contactsensor_state/states[0]/contact_normals ==> NormalContact in World

        ==> One is an array of all the forces, the other total,
         and are relative to the contact link referred to in the sensor.
        /lowerleg_contactsensor_state/states[0]/wrenches[]
        /lowerleg_contactsensor_state/states[0]/total_wrench
        :return:
        """
        
        # We create an empty element , in case there is no contact.
        contact_force = Vector3()
        for state in lowerleg_contactsensor_state.states:
            self.contact_force = state.total_wrench.force
        
        return contact_force
        
        
    def monoped_height_ok(self, height_base):

        height_ok = self.min_height <= height_base < self.max_height
        return height_ok
        
    def monoped_orientation_ok(self):

        orientation_rpy = self.get_base_rpy()
        roll_ok = self.max_incl_roll > abs(orientation_rpy.x)
        pitch_ok = self.max_incl_pitch > abs(orientation_rpy.y)
        orientation_ok = roll_ok and pitch_ok
        return orientation_ok
        
        
    def calculate_reward_joint_position(self, joints_state_array, weight=1.0):
        """
        We calculate reward base on the joints configuration. The more near 0 the better.
        :return:
        """
        acumulated_joint_pos = 0.0
        for joint_pos in joints_state_array:
            # Abs to remove sign influence, it doesnt matter the direction of turn.
            acumulated_joint_pos += abs(joint_pos)
            rospy.logdebug("calculate_reward_joint_position>>acumulated_joint_pos=" + str(acumulated_joint_pos))
        reward = weight * acumulated_joint_pos
        rospy.logdebug("calculate_reward_joint_position>>reward=" + str(reward))
        return reward
        
    def calculate_reward_contact_force(self, force_magnitude, weight=1.0):
        """
        We calculate reward base on the contact force.
        The nearest to the desired contact force the better.
        We use exponential to magnify big departures from the desired force.
        Default ( 7.08 N ) desired force was taken from reading of the robot touching
        the ground from a negligible height of 5cm.
        :return:
        """
        force_displacement = force_magnitude - self.desired_force

        rospy.logdebug("calculate_reward_contact_force>>force_magnitude=" + str(force_magnitude))
        rospy.logdebug("calculate_reward_contact_force>>force_displacement=" + str(force_displacement))
        # Abs to remove sign
        reward = weight * abs(force_displacement)
        rospy.logdebug("calculate_reward_contact_force>>reward=" + str(reward))
        return reward
        
    def calculate_reward_orientation(self, rpy_array, weight=1.0):
        """
        We calculate the reward based on the orientation.
        The more its closser to 0 the better because it means its upright
        desired_yaw is the yaw that we want it to be.
        to praise it to have a certain orientation, here is where to set it.
        :param: rpy_array: Its an array with Roll Pitch and Yaw in place 0, 1 and 2 respectively.
        :return:
        """

        yaw_displacement = rpy_array[2] - self.desired_yaw
        acumulated_orientation_displacement = abs(rpy_array[0]) + abs(rpy_array[1]) + abs(yaw_displacement)
        reward = weight * acumulated_orientation_displacement
        rospy.logdebug("calculate_reward_orientation>>reward=" + str(reward))
        return reward
        
    def calculate_reward_distance_from_des_point(self, current_position, weight=1.0):
        """
        We calculate the distance from the desired point.
        The closser the better
        :param weight:
        :return:reward
        """
        distance = self.get_distance_from_desired_point(current_position)
        reward = weight * distance
        rospy.logdebug("calculate_reward_orientation>>reward=" + str(reward))
        return reward

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Python File: hopper_task_env.py** </p>

## Solution Step 3: Create Training Script (Qlearn) <p id="SolutionStep3"></p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Python File: start_training.py** </p>

In [ ]:
#!/usr/bin/env python

import gym
import numpy
import time
import qlearn
from gym import wrappers
# ROS packages required
import rospy
import rospkg
# import our training environment
import hopper_task_env


if __name__ == '__main__':

    rospy.init_node('hopper_stay_up_qlearn', anonymous=True, log_level=rospy.WARN)

    # Create the Gym environment
    env = gym.make('HopperStayUp-v0')
    rospy.loginfo("Gym environment done")

    # Set the logging system
    rospack = rospkg.RosPack()
    pkg_path = rospack.get_path('my_hopper_openai_example')
    outdir = pkg_path + '/training_results'
    env = wrappers.Monitor(env, outdir, force=True)
    rospy.loginfo("Monitor Wrapper started")

    last_time_steps = numpy.ndarray(0)

    # Loads parameters from the ROS param server
    # Parameters are stored in a yaml file inside the config directory
    # They are loaded at runtime by the launch file
    Alpha = rospy.get_param("/monoped/alpha")
    Epsilon = rospy.get_param("/monoped/epsilon")
    Gamma = rospy.get_param("/monoped/gamma")
    epsilon_discount = rospy.get_param("/monoped/epsilon_discount")
    nepisodes = rospy.get_param("/monoped/nepisodes")
    nsteps = rospy.get_param("/monoped/nsteps")

    # Initialises the algorithm that we are going to use for learning
    qlearn = qlearn.QLearn(actions=range(env.action_space.n),
                           alpha=Alpha, gamma=Gamma, epsilon=Epsilon)
    initial_epsilon = qlearn.epsilon

    start_time = time.time()
    highest_reward = 0

    # Starts the main training loop: the one about the episodes to do
    for x in range(nepisodes):
        rospy.logdebug("############### START EPISODE=>" + str(x))

        cumulated_reward = 0
        done = False
        if qlearn.epsilon > 0.05:
            qlearn.epsilon *= epsilon_discount

        # Initialize the environment and get first state of the robot
        observation = env.reset()
        state = ''.join(map(str, observation))

        # Show on screen the actual situation of the robot
        # env.render()
        # for each episode, we test the robot for nsteps
        for i in range(nsteps):
            rospy.logwarn("############### Start Step=>" + str(i))
            # Pick an action based on the current state
            action = qlearn.chooseAction(state)
            rospy.logwarn("Next action is:%d", action)
            # Execute the action in the environment and get feedback
            observation, reward, done, info = env.step(action)

            rospy.logwarn(str(observation) + " " + str(reward))
            cumulated_reward += reward
            if highest_reward < cumulated_reward:
                highest_reward = cumulated_reward

            nextState = ''.join(map(str, observation))

            # Make the algorithm learn based on the results
            rospy.logwarn("# state we were=>" + str(state))
            rospy.logwarn("# action that we took=>" + str(action))
            rospy.logwarn("# reward that action gave=>" + str(reward))
            rospy.logwarn("# episode cumulated_reward=>" + str(cumulated_reward))
            rospy.logwarn("# State in which we will start next step=>" + str(nextState))
            qlearn.learn(state, action, reward, nextState)

            if not (done):
                rospy.logwarn("NOT DONE")
                state = nextState
            else:
                rospy.logwarn("DONE")
                last_time_steps = numpy.append(last_time_steps, [int(i + 1)])
                break
            rospy.logwarn("############### END Step=>" + str(i))
            #raw_input("Next Step...PRESS KEY")
            # rospy.sleep(2.0)
        m, s = divmod(int(time.time() - start_time), 60)
        h, m = divmod(m, 60)
        rospy.logerr(("EP: " + str(x + 1) + " - [alpha: " + str(round(qlearn.alpha, 2)) + " - gamma: " + str(
            round(qlearn.gamma, 2)) + " - epsilon: " + str(round(qlearn.epsilon, 2)) + "] - Reward: " + str(
            cumulated_reward) + "     Time: %d:%02d:%02d" % (h, m, s)))

    rospy.loginfo(("\n|" + str(nepisodes) + "|" + str(qlearn.alpha) + "|" + str(qlearn.gamma) + "|" + str(
        initial_epsilon) + "*" + str(epsilon_discount) + "|" + str(highest_reward) + "| PICTURE |"))

    l = last_time_steps.tolist()
    l.sort()

    # print("Parameters: a="+str)
    rospy.loginfo("Overall score: {:0.2f}".format(last_time_steps.mean()))
    rospy.loginfo("Best 100 score: {:0.2f}".format(reduce(lambda x, y: x + y, l[-100:]) / len(l[-100:])))

    env.close()

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Python File: start_training.py** </p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Param File: hopper_qlearn_params.yaml** </p>

In [ ]:
monoped: #namespace

    #qlearn parameters
    alpha: 0.1
    gamma: 0.7
    epsilon: 0.9
    epsilon_discount: 0.999
    nepisodes: 500
    nsteps: 10000

    n_actions: 6 # 1-2) Increment/Decrement haa_joint
                 # 3-4) Increment/Decrement hfe_joint
                 # 5-6) Increment/Decrement kfe_joint
                 
    n_observations: 6 # We have 6 different observations


    init_joint_states:
      haa_joint: 0.0
      hfe_joint: 0.0
      kfe_joint: 0.0

    desired_point:
      x: 0.0
      y: 0.0
      z: 1.0
    accepted_error_in_des_pos: 0.1 # Accepted error un meters of the desired position
      
    number_decimals_precision_obs: 1
      
    desired_force: 7.08 # In Newtons, normal contact force when stanting still with 9.81 gravity
    max_contact_force: 14.16 # The maxixmum contact force that we consoder acceptable.
    
    desired_yaw: 0.0 # Desired yaw in radians for the hopper to stay
    
    max_x_pos: 3.0 # Maximum Position of the base in X axis in World
    max_y_pos: 3.0 # Maximum Position of the base in Y axis in World
    
    max_height: 3.0   # in meters
    min_height: 0.5   # in meters
    
    max_incl: 1.57       # Maximum acceptable roll and pitch for the hoppers base. More is conidered a fallen robot
    
    distance_from_desired_point_max: 3.0 # Distance that we consider maximum to be from the desired point
    
    joint_increment_value: 0.05  # in radians
    accepted_joint_error: 0.1 # Radians of accepted error in the joints, should be bigger than the joint increment
    update_rate: 20 # Hz frequency in whihc we check that the joints are in the correct place, the bigger the more precise but more overhead
    
    maximum_haa_joint: 1.6 # Max and Min value ( sign change ) based on the URDF joint values
    maximum_hfe_joint: 1.6 # Max and Min value ( sign change ) based on the URDF joint values
    maximum_kfe_joint: 0.0 # Max based on the URDF joint values
    min_kfe_joint: -1.6 #  Min value based on the URDF joint values
    
    done_reward: 1000.0 # reward
    alive_reward: 100.0 # reward
    
    rewards_weight:
      weight_joint_position: 1.0 # Weight for joint positions ( joints in the zero is perfect )
      weight_contact_force: 1.0 # Weight for contact force similar to desired ( weight of monoped )
      weight_orientation: 1.0 # Weight for orientation ( vertical is perfect )
      weight_distance_from_des_point: 1.0 # Weight for distance from desired point ( on the point is perfect )

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Param File: hopper_qlearn_params.yaml** </p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Python File: qlearn.py** </p>

In [ ]:
'''
Q-learning approach for different RL problems
as part of the basic series on reinforcement learning @
https://github.com/vmayoral/basic_reinforcement_learning
 
Inspired by https://gym.openai.com/evaluations/eval_kWknKOkPQ7izrixdhriurA
 
        @author: Victor Mayoral Vilches <victor@erlerobotics.com>
'''
 
import random
 
class QLearn:
    def __init__(self, actions, epsilon, alpha, gamma):
        self.q = {}
        self.epsilon = epsilon  # exploration constant
        self.alpha = alpha      # discount constant
        self.gamma = gamma      # discount factor
        self.actions = actions
 
    def getQ(self, state, action):
        return self.q.get((state, action), 0.0)
 
    def learnQ(self, state, action, reward, value):
        '''
        Q-learning:
            Q(s, a) += alpha * (reward(s,a) + max(Q(s') - Q(s,a))            
        '''
        oldv = self.q.get((state, action), None)
        if oldv is None:
            self.q[(state, action)] = reward
        else:
            self.q[(state, action)] = oldv + self.alpha * (value - oldv)
 
    def chooseAction(self, state, return_q=False):
        q = [self.getQ(state, a) for a in self.actions]
        maxQ = max(q)
 
        if random.random() < self.epsilon:
            minQ = min(q); mag = max(abs(minQ), abs(maxQ))
            # add random values to all the actions, recalculate maxQ
            q = [q[i] + random.random() * mag - .5 * mag for i in range(len(self.actions))] 
            maxQ = max(q)
 
        count = q.count(maxQ)
        # In case there're several state-action max values 
        # we select a random one among them
        if count > 1:
            best = [i for i in range(len(self.actions)) if q[i] == maxQ]
            i = random.choice(best)
        else:
            i = q.index(maxQ)
 
        action = self.actions[i]        
        if return_q: # if they want it, give it!
            return action, q
        return action
 
    def learn(self, state1, action1, reward, state2):
        maxqnew = max([self.getQ(state2, a) for a in self.actions])
        self.learnQ(state1, action1, reward, reward + self.gamma*maxqnew)

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Python File: qlearn.py** </p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Launch File: start_training.launch** </p>

In [ ]:
<launch>
 
    <!-- Load the parameters for the algorithm -->
    <rosparam command="load" file="$(find hopper_project_pkg)/config/hopper_qlearn_params.yaml" />
 
    <!-- Launch the training system -->
    <node pkg="hopper_project_pkg" name="hopper_stay_up_qlearn" type="start_training.py" output="screen"/>
</launch>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Launch File: start_training.launch** </p>